In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import keras
from keras.layers import Dense, Dropout, BatchNormalization
from keras.models import Sequential
from sklearn.metrics import brier_score_loss, accuracy_score
from keras.utils import to_categorical

def decodePhed(x):
    return 1-10**(-x/10.0)

/public/home/jchan67/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
train_file = "data/train_gamcompare/json/compared_mapped100_sim100.json"
test_file = "data/test_gamcompare/json/tcompared_tmapped100_tsim100.json"
model_file = "data/models/model_len100.h5"

In [6]:
def json2csv(file):
    df_dict = {
        'correct': list(),
        'mq': list(),
        'score': list(),
        'secondary_score' : list(),
        'secondary_score_size':list(),
        'identity': list(),
        'aligner' : list(),
        'read': list()
    }
    with open(file, "r+") as f:
        line = f.readline()
        #line_dict = json.loads(line) 
        i = 0
        while(line != ""):
            line_dict = json.loads(line)
            if 'correctly_mapped' in line_dict:
                df_dict['correct'].append(1)
            else:
                df_dict['correct'].append(0)
                
            if 'mapping_quality' in line_dict:
                df_dict['mq'].append(line_dict['mapping_quality'])
            else:
                df_dict['mq'].append(0)
            
            if 'score' in line_dict:
                df_dict['score'].append(line_dict['score'])
            else:
                df_dict['score'].append(0)
                
            if 'identity' in line_dict:
                df_dict['identity'].append(line_dict['identity'])
            else:
                df_dict['identity'].append(0)
                
            if 'secondary_score' in line_dict:
                df_dict['secondary_score'].append(line_dict['secondary_score'][0])
                df_dict['secondary_score_size'].append(len(line_dict['secondary_score']))
            else:
                df_dict['secondary_score'].append(0)
                df_dict['secondary_score_size'].append(0)
            
            df_dict['aligner'].append('orig')
            df_dict['read'].append(line_dict['name'])
            
            line = f.readline()
            i += 1
        print(i)

    return pd.DataFrame(df_dict)
    
df = json2csv(test_file)

1000000


In [8]:
tsv_data = df.copy()
tsv_data = tsv_data.drop(['score', 'secondary_score', 'identity', 'secondary_score_size'], axis=1)

In [9]:
ndf = df.copy()

In [10]:
ndf['mq'] = df.mq/60.0
ndf['score'] = df.score/df.score.max()
ndf['secondary_score'] = df.secondary_score/df.secondary_score.max()
ndf['secondary_score_size'] = df.secondary_score_size/df.secondary_score_size.max()

In [11]:
ndf.head()

,correct,mq,score,secondary_score,secondary_score_size,identity,aligner,read
0,1,0.25,0.954545,0.909091,0.007874,0.99,orig,86fbc8512647802d
1,1,1.00,0.954545,0.000000,0.000000,0.99,orig,2fb1922aea224dbe
2,1,1.00,0.954545,0.000000,0.007874,0.99,orig,93dfa21d81013e2b
3,1,1.00,0.954545,0.000000,0.000000,0.99,orig,57e5a50ad9763655
4,1,1.00,1.000000,0.000000,0.000000,1.00,orig,2a016082a1d0a262


In [12]:
incorrect_amount = ndf[ndf.correct == 0]['correct'].count()
incorrect_amount = 1000000

In [13]:
train_incorrect_amount = int(incorrect_amount * 0.8)
test_incorrect_amount = int(incorrect_amount * 0.2)
train_incorrect_amount,test_incorrect_amount 

(800000, 200000)

In [14]:
X = ndf.iloc[:, 1:6]
y = ndf.iloc[:, 0]


In [15]:
permu_index = np.random.permutation(X.shape[0])

In [16]:
X = X.iloc[permu_index, :]
y = y.iloc[permu_index]

In [122]:
# X_train_correct = X[y.correct == 1].iloc[:train_incorrect_amount]
# y_train_correct = y[y.correct == 1].iloc[:train_incorrect_amount]

# X_train_incorrect = X[y.correct == 0].iloc[:train_incorrect_amount]
# y_train_incorrect = y[y.correct == 0].iloc[:train_incorrect_amount]

# X_test_correct = X[y.correct == 1].iloc[train_incorrect_amount:train_incorrect_amount+test_incorrect_amount]
# y_test_correct = y[y.correct == 1].iloc[train_incorrect_amount:train_incorrect_amount+test_incorrect_amount]

# X_test_incorrect = X[y.correct == 0].iloc[train_incorrect_amount:train_incorrect_amount+test_incorrect_amount]
# y_test_incorrect = y[y.correct == 0].iloc[train_incorrect_amount:train_incorrect_amount+test_incorrect_amount]


# print(X_train_correct.shape)
# print(X_train_incorrect.shape)
# print(X_test_correct.shape)
# print(X_test_incorrect.shape)


In [123]:
# X_train = np.append(X_train_correct, X_train_incorrect, axis=0)
# X_test = np.append(X_test_correct, X_test_incorrect, axis=0)

# y_train = np.append(y_train_correct, y_train_incorrect)
# y_test = np.append(y_test_correct, y_test_incorrect)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(800000, 5)
(200000, 5)
(800000, 1)
(200000, 1)


In [124]:
X_train = X[:train_incorrect_amount].values
X_test = X[train_incorrect_amount:train_incorrect_amount+test_incorrect_amount].values

y_train = y[:train_incorrect_amount].values
y_test = y[train_incorrect_amount:train_incorrect_amount+test_incorrect_amount].values

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(800000, 5)
(200000, 5)
(800000,)
(200000,)


In [125]:
y_train_class = np.zeros((800000, 2))

y_train_class[(y_train==True), : ] = [1, 0]
y_train_class[(y_train==False), :] = [0, 1]

In [126]:
y_train

array([1, 1, 1, ..., 1, 1, 1])

In [127]:
y_test_class = np.zeros((200000, 2))

y_test_class[(y_test==True), : ] = [1, 0]
y_test_class[(y_test==False), :] = [0, 1]

In [128]:
y_train_class, y_test_class

(array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]), array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]))

#### Neural Network HyperParameters

In [129]:
input_layer = 5
output_layer = 2 

h_layer1 = 8
dropout1 = 0.25

h_layer2 = 16
dropout2 = 0.5

h_layer3 = 16
dropout3 = 0.5

h_layer4 = 8
dropout4 = 0.5

#### Neural Network Architecture

In [130]:
model = Sequential()

model.add(Dense(h_layer1, activation='relu', input_shape=(input_layer, )))
model.add(BatchNormalization())
model.add(Dropout(dropout1))

model.add(Dense(h_layer2, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout2))

model.add(Dense(h_layer3, activation='relu'))
model.add(Dropout(dropout3))

model.add(Dense(h_layer4, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout4))

model.add(Dense(output_layer, activation='softmax'))

In [131]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 8)                 48        
_________________________________________________________________
batch_normalization_4 (Batch (None, 8)                 32        
_________________________________________________________________
dropout_5 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 16)                144       
_________________________________________________________________
batch_normalization_5 (Batch (None, 16)                64        
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 16)                272       
__________

In [132]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train_class, batch_size=128, epochs=30, 
          verbose=1, validation_data=(X_test, y_test_class), shuffle=True)

Train on 800000 samples, validate on 200000 samples
Epoch 1/30
800000/800000 [==============================] - 311s 389us/step - loss: 0.0082 - acc: 0.9975 - val_loss: 0.0054 - val_acc: 0.9979
Epoch 2/30
800000/800000 [==============================] - 274s 342us/step - loss: 0.0070 - acc: 0.9975 - val_loss: 0.0046 - val_acc: 0.9979
Epoch 3/30
800000/800000 [==============================] - 267s 333us/step - loss: 0.0069 - acc: 0.9975 - val_loss: 0.0050 - val_acc: 0.9979
Epoch 4/30
800000/800000 [==============================] - 298s 372us/step - loss: 0.0065 - acc: 0.9975 - val_loss: 0.0052 - val_acc: 0.9979
Epoch 5/30
800000/800000 [==============================] - 471s 588us/step - loss: 0.0065 - acc: 0.9975 - val_loss: 0.0044 - val_acc: 0.9979
Epoch 6/30
800000/800000 [==============================] - 478s 598us/step - loss: 0.0062 - acc: 0.9977 - val_loss: 0.0041 - val_acc: 0.9980
Epoch 7/30
800000/800000 [==============================] - 321s 401us/step - loss: 0.0060 - acc

In [134]:
X_sample = X_test[y_test == 0][0:100]

model.predict(X_sample)

array([[6.54491484e-01, 3.45508575e-01],
       [5.20208895e-01, 4.79791075e-01],
       [8.80289793e-01, 1.19710207e-01],
       [7.98819721e-01, 2.01180279e-01],
       [6.43149555e-01, 3.56850445e-01],
       [1.19745696e-03, 9.98802543e-01],
       [8.79633129e-01, 1.20366871e-01],
       [7.99816847e-01, 2.00183153e-01],
       [6.83864534e-01, 3.16135406e-01],
       [2.55335718e-02, 9.74466503e-01],
       [8.80943120e-01, 1.19056851e-01],
       [6.52612805e-01, 3.47387195e-01],
       [5.57245910e-01, 4.42754149e-01],
       [6.46948934e-01, 3.53051096e-01],
       [8.79633129e-01, 1.20366871e-01],
       [8.01799834e-01, 1.98200166e-01],
       [7.79170334e-01, 2.20829666e-01],
       [8.13386917e-01, 1.86613068e-01],
       [8.82240772e-01, 1.17759302e-01],
       [4.96164605e-04, 9.99503851e-01],
       [3.39533150e-01, 6.60466850e-01],
       [8.64387214e-01, 1.35612816e-01],
       [8.80289793e-01, 1.19710207e-01],
       [8.83526206e-01, 1.16473794e-01],
       [6.450517

In [147]:
model.save('model_2-all.h5')
model.save_weights('model_2-w-all.h5')

In [135]:
y_pred = model.predict(ndf.iloc[:, 1:6].values) 

In [136]:
orig = decodePhed(df.mq.values)
labels = df.correct.values
nn_pred = y_pred[:, 0]

In [137]:
indx = ((y_pred[:, 0]  >= 0.90) | (y_pred[:, 0]  <= 0.10))

In [138]:
nn_pred[indx]

array([0.99993706, 0.9999318 , 0.99993706, ..., 0.9999306 , 0.99993706,
       0.99995637], dtype=float32)

In [139]:
brier_score_loss(labels[indx], nn_pred[indx])

0.0001320709796901863

In [140]:
brier_score_loss(labels, orig) 

0.0017481454421636765

In [141]:
nn_pred[nn_pred >= 0.5] = 1
nn_pred[nn_pred < 0.5] = 0

In [142]:
accuracy_score(labels, nn_pred)

0.997873

In [143]:
orig[orig >= 0.5] = 1
orig[orig < 0.5] = 0

In [144]:
accuracy_score(labels, orig)

0.998082

In [145]:
accuracy_score(labels[labels == 1], nn_pred[labels == 1])

0.9996822294063104

In [146]:
accuracy_score(labels[labels == 1], orig[labels == 1])

0.9983239355436935

In [148]:
accuracy_score(labels[labels == 0], nn_pred[labels == 0])

0.2536082474226804

In [149]:
accuracy_score(labels[labels == 0], orig[labels == 0])

0.8985567010309279